# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.model import InferenceConfig
from azureml.core import Environment
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'udacity-project-automl'
project_folder = './automl-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
udacity-project-automl,quick-starts-ws-251798,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "udacity-finalproject-compute"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Check if the compute cluster already exists, if not, create a new one
if cluster_name not in ws.compute_targets:
    # Configure the provisioning details for the compute cluster
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2s_v3', max_nodes=4)

    # Create the compute cluster
    compute_target = ComputeTarget.create(ws, cluster_name, provisioning_config)

    # Wait for the compute cluster to be provisioned
    compute_target.wait_for_completion(show_output=True)
else:
    compute_target = ws.compute_targets[cluster_name]

In [4]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "HeartFailurePrediction"
description_text = "Heart Failure Prediction Dataset Kaggle"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:      
        # Set Kaggle credentials as environment variables or in a separate Kaggle.json file  
        import os  
        os.environ['KAGGLE_USERNAME'] = '<your_kaggle_username>'  
        os.environ['KAGGLE_KEY'] = '<your_kaggle_api_key>'  

        # kaggle datasets download -d fedesoriano/heart-failure-prediction
        
        # Download the Kaggle dataset using the KaggleDatasetFactory  
        from azureml.data.dataset_factory import KaggleDatasetFactory  

        factory = KaggleDatasetFactory()  
        dataset = factory.create_dataset(  
            dataset_id='<kaggle_dataset_id>',  
            version='<kaggle_dataset_version>'  
        )  

        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="HeartDisease",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )



In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-automl,AutoML_0b5314a5-ef77-4c1d-9fee-1228a8a8b3aa,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [29]:
# Get the best model  
best_run, fitted_model = remote_run.get_output()  
# print(best_run)
# print(best_run.get_file_names())
best_run.download_file(name='outputs/model.pkl')

model_name = 'AM_best_model'  
model_description = 'AM model with best accuracy'  
tags = {'accuracy': best_run.get_metrics()['accuracy']}  


registered_model = Model.register(workspace=ws,  
                                 model_path='model.pkl',
                                 model_name=model_name,  
                                 description=model_description,  
                                 tags=tags)  


Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

['accuracy_table', 'automl_driver.py', 'confusion_matrix', 'explanation/76d98acc/classes.interpret.json', 'explanation/76d98acc/eval_data_viz.interpret.json', 'explanation/76d98acc/expected_values.interpret.json', 'explanation/76d98acc/features.interpret.json', 'explanation/76d98acc/global_names/0.interpret.json', 'explanation/76d98acc/global_rank/0.interpret.json', 'explanation/76d98acc/global_values/0.interpret.json', 'explanation/76d98acc/local_importance_values.interpret.json', 'explanation/76d98acc/per_class_names/0.interpret.json', 'explanation/76d98acc/per_class_rank/0.interpret.json', 'explanation/76d98acc/per_class_values/0.interpret.json', 'explanation/76d98acc/rich_metadata.interpret.json', 'explanation/76d98acc/true_ys_viz.interpret.json', 'explanation/76d98acc/visualization_dict.interpret.json', 'explanation/76d98acc/ys_pred_proba_viz.interpret.json', 'explanation/76d98acc/ys_pred_viz.interpret.json', 'explanation/99e6e5c2/classes.interpret.json', 'explanation/99e6e5c2/exp

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [32]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
myenv = Environment.from_conda_specification(name='myenv', file_path='myenv.yml')
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inf_score.py')

inference_config = InferenceConfig(entry_script='inf_score.py', environment=myenv)  
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)  

service_name = 'automl-model-service'  
  
service = Model.deploy(workspace=ws,  
                       name=service_name,  
                       models=[registered_model],  
                       inference_config=inference_config,  
                       deployment_config=deployment_config)  
  
service.wait_for_deployment(show_output=True)  

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-06 04:57:04+00:00 Creating Container Registry if not exists.
2024-02-06 04:57:05+00:00 Registering the environment.
2024-02-06 04:57:07+00:00 Building image..
2024-02-06 05:47:33+00:00 Generating deployment configuration.
2024-02-06 05:47:34+00:00 Submitting deployment to compute..
2024-02-06 05:47:40+00:00 Checking the status of deployment automl-model-service..
2024-02-06 05:50:28+00:00 Checking the status of inference endpoint automl-model-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [40]:
# Get the deployed web service  
from azureml.core.webservice import Webservice
import json
service_name = 'automl-model-service'  
service = Webservice(workspace=ws, name=service_name)  
  
# Load the input data for making predictions  
data =  {
  "data": [
    {
      "Age": 57,
      "Sex": "M",
      "ChestPainType": "ATA",
      "RestingBP": 140,
      "Cholesterol": 289,
      "FastingBS": 0,
      "RestingECG": "Normal",
      "MaxHR": 172,
      "ExerciseAngina": "false",
      "Oldpeak": 0.0,
      "ST_Slope": "up"
    }
  ],
  "method": "predict"
}
  
# Convert the data to JSON string  
input_data = json.dumps(data)  
  
# Call the web service to make predictions  
result = service.run(input_data)  

print(result)
# Print the predictions  
# predictions = json.loads(result)['predictions']  
# for prediction in predictions:  
#     print(prediction) 

{"result": [0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [41]:
# Print the logs of the web service  
logs = service.get_logs()  
print(logs)

# Delete the web service  
service.delete()  

/bin/bash: /azureml-envs/azureml_54d5b9858d4d11617085aab4b1be851c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_54d5b9858d4d11617085aab4b1be851c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_54d5b9858d4d11617085aab4b1be851c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2024-02-06T05:49:46,329089700+00:00 - rsyslog/run 
2024-02-06T05:49:46,334590700+00:00 - gunicorn/run 
2024-02-06T05:49:46,344323400+00:00 | gunicorn/run | 
2024-02-06T05:49:46,346098700+00:00 | gunicorn/run | ###############################################
2024-02-06T05:49:46,351096800+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-02-06T05:49:46,353685500+00:00 | gunicorn/run | ###############################################
2024-02-06T05:49:46,364549400+00:00 | gunicorn/run | 
bash: /azureml-envs/azureml_54d5b9858d4d11617085aab4b1be851c/lib/libti

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
